In [1]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
import logging
from selenium.common.exceptions import TimeoutException, WebDriverException

In [29]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')
# 키워드 입력
search_box.send_keys("종로구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')
    load_more_button.click()
    time.sleep(3)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))
# 스크롤 올리기
driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

페이지에 표시된 정보의 갯수: 76


### 가게 이름 클릭 함수 다시만들기 - class 이용

In [25]:
for index in range(2):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(2)
        # 가게 이름 크롤링
        cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
        # 후기 갯수 크롤링
        qa = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]').text
        store = [cw for _ in range(int(qa))]
        time.sleep(3)
        driver.back()
    except NoSuchElementException:
        continue
    


In [28]:
len(store)

224

In [ ]:
# class 이용 가게 함수 클릭- 실패
# def store_click():
strong_classes = driver.find_elements(By.CLASS_NAME, 'tit_g')
for index in range(3):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    driver.find_element(By.CLASS_NAME, 'tit_g').find_element(By.XPATH, xpath).click()
    time.sleep(2)
    driver.back()
    
# store_click()

# 가게 이름 누르고 후기 더보기 누르기

1. 가게 이름 누르고 들어간다
2. 가게 이름 크롤링
3. rate_intro 찾아서 있으면, 4번, 없으면, 7번
4. 첫번째 후기 클릭해서 전체 후기 들어가기
5. 전체 후기 페이지의 더보기 끝 까지 누르기 
6. 뒤로가기 2번
7. 없으면, 뒤로가기 1번

In [30]:
# 연습 1-4 완료

# 랜덤한 대기 시간
wait = random.uniform(5,10)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

            try:
                id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                temp_ID = id_element.text.strip()
                if temp_ID:
                    ID.append(id_element.text)
                else:
                    ID.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                ID.append('정보 없음')
                # break
            # ID.pop()
            time.sleep(wait)

            try:
                date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                Date.append(date_element.text)
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                break

            time.sleep(wait)

            try:
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()
                if temp_review_text:
                    Review_text.append(temp_review_text)
                else:
                    Review_text.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException):  
                Review_text.append('정보 없음')

            driver.execute_script("window.scrollBy(0, 200);")
            g += 1
            attempts = 0
        # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
        except (WebDriverException or NoSuchElementException or TimeoutException):  
            attempts += 1

        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break
        
# 연습 1-4 완료

In [34]:
# 연습 2번 완료
ID = []
Date = []
Review_text = []
# for i in range(5, len(all_results) + 1):
for index in range(26, 27):
    wait_time = random.uniform(1, 5)
    if NoSuchElementException:
        click_load_more_button()
    # 가게 이름 클릭
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(wait_time)
        # 가게 이름 크롤링
        cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
        # 후기 갯수 크롤링
        qa = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]').text
        store = [cw for _ in range(int(qa))]
        time.sleep(wait_time)
        # driver.back()
    except NoSuchElementException:
        continue
    
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 첫 번째 더보기 버튼 클릭
    class_click()
 
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    # 크롤링 함수
    find_data()
    ID.pop()
    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)
    # # 더보기 버튼
    # if can_click_load_more_button():
    #     click_load_more_button()
    # else :
    #     continue

Class changed. Stopping click.


In [199]:
oh = [temp_store[0] for _ in range(36)]
k = [temp_store[1] for _ in range(57)]
wer = [temp_store[2] for _ in range(56)]
kl = [temp_store[3] for _ in range(45)]
jk = [temp_store[4] for _ in range(65)]
store = oh + k + wer + kl + jk
len(store)

259

In [38]:
temp_store

['정애쿠키']

In [35]:
print(len(store))
print(len(ID))
print(len(Date))
print(len(Review_text))

25
25
25
25


In [184]:
temp_store

['온혜화']

In [36]:
jo25 = pd.DataFrame()
jo25['Store'] = stor
jo25['ID'] = ID
jo25['Date'] = Date
jo25['Review_text'] = Review_text
jo25

,Store,ID,Date,Review_text
0,정애쿠키,alwodnn,2024.03.30.,사장님께서 정말 친절하세요.\n기분 좋아지는 웃음과 친절한 서비스까지 너무 좋았습니...
1,정애쿠키,민구♣,2024.02.14.,정보 없음
2,정애쿠키,붕어빵,2024.01.30.,쿠키는 다이제 오리지날 느낌나요ㅎㅎ 통밀 좋아하시는 분들은 좋아할실듯. 개인적으론 ...
3,정애쿠키,ash,2024.01.06.,정보 없음
4,정애쿠키,.,2023.12.20.,고추쿠키 정말 맛있어요 👍
5,정애쿠키,준영,2023.12.03.,"일본 만화에 나올 것만 같은,, 너무 친절하시고 따듯하신 사장님 마음씨가 커피와 쿠..."
6,정애쿠키,연님이,2023.11.17.,정보 없음
7,정애쿠키,부엉이,2023.11.05.,"사장님 너무 좋으시고, 쿠키도 정말 맛있어요. 올때마다 사갑니다.\n\n정애쿠키(오..."
8,정애쿠키,Silver_Phot...,2023.08.27.,할머니가 정말 밝고 친절하시고 좋았어요.\n쿠키도 버터없이 통밀가루와 아몬드 등등 ...
9,정애쿠키,맛있는거먹으면짖는개,2023.08.11.,정보 없음


In [202]:
total_jong = pd.concat([total_jong, jo2024], ignore_index=True)
total_jong

,Store,ID,Date,Review_text
0,스코프 서촌점,시리얼,2024.03.30.,예전에 스콘 먹고 감동했던 기억이 있어 다시 방문했는데 조금 실망입니다. 너무 달아...
1,스코프 서촌점,쩝문가,2024.03.23.,명성에 비해 스콘이나 타 디저트 애매합니다. \n유명하지 않았던 평타라 생각하지만\...
2,스코프 서촌점,박현아,2024.03.01.,정보 없음
3,스코프 서촌점,별점조작은말없이1점,2024.02.25.,갠적으로 스콘은 질기고 걍평범하고 맛없고 아아 싫어서 차시켰는데 차도 너무 걍 싸구...
4,스코프 서촌점,고영,2024.02.20.,다 맛있다
...,...,...,...,...
2513,아티장크로아상,sorasun,2019.04.21.,정보 없음
2514,아티장크로아상,beentheredo...,2019.04.13.,정보 없음
2515,아티장크로아상,맛찾아삼만리,2019.03.26.,정보 없음
2516,아티장크로아상,새별,2019.03.03.,바삭하고 눅진한 크라상이랑 따끈한 커피한잔 먹으면 그냥 그렇게 좋더라.


In [37]:
jo25.to_csv('jo25.csv')